### Pre-procesamiento de los datos

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

"""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""

data=pd.read_csv("../input/heart-disease-prediction-using-logistic-regression/framingham.csv") 
data.head() 

In [ ]:
# exploramos la forma del data set
data.shape

In [ ]:
data.dtypes

In [ ]:
data.describe()

In [ ]:
# verificamos si hay valores nulos con la funcion isnull
pd.isnull(data["male"])

In [ ]:
# glucose tiene valores nulos, con las funciones ravel y sum 
pd.isnull(data["glucose"]).ravel().sum()

In [ ]:
# tambien podemos usar esta funcion para hallar los valores nulos
len(data[pd.isnull(data.glucose)]) # 388
# como es un tipo de dato que nos dice los niveles de glucosa lo mejor seria rellenarlos con el promedio con la funcion fillna
data["glucose"]=data["glucose"].fillna(data["glucose"].mean()) 
# comprobamos que elos valores nulos del data set ahora son cero 
len(data[pd.isnull(data.glucose)])

In [ ]:
# observamos la cantidad de datos nulos en educacion y como no tiene descripción y no es muy importante para el caso de estudio la eliminaremos 
pd.isnull(data["education"]).ravel().sum() 
data=data.drop(["education"],axis=1) 
data.head()

In [ ]:
# la variable cigsPerDay tambien presenta datos faltantes, por la naturaleza los remplazaremos con el promedio
data["cigsPerDay"]=data["cigsPerDay"].fillna(data["cigsPerDay"].mean())
pd.isnull(data["cigsPerDay"]).ravel().sum() #conprobamos que ya no haya valores faltantes

In [ ]:
# como la varible BPmeds puede tener una gran significancia y solo son 53 observaciones eliminaremos los valores nulos del dataset
# eliminaremos las filas con valores faltantes de la columnas BPMeads #https://interactivechaos.com/es/python/scenario/eliminacion-de-filas-en-un-dataframe-pandas-partir-de-una-condicion
pd.isnull(data["BPMeds"]).ravel().sum()
df = data[data.BPMeds.isnull()] #este proceso se llma indexar para que con drop podamos eliminar solo los valores nulos de una fila
df.shape # aqui vermos el dataset con los valores nulos
df.index # optenemos el indice
data.drop(data[data.BPMeds.isnull()].index, inplace = True) # aqui ya usamos drop y elimonamos las filas con na de la columna BPmeds
data.shape # vemos la nueva cantidad de datos

In [ ]:
# la variable toChol al ser los niveles de colesterol, continua remplazaremos los faltantes con la media
data["totChol"]= data["totChol"].fillna(data["totChol"].mean())
pd.isnull(data["totChol"]).ravel().sum()

In [ ]:
# BMI de igual manera al ser el indice de masa corporal lo remplazamos por la media

data["BMI"]=data["BMI"].fillna(data["BMI"].mean()) 
pd.isnull(data["BMI"]).ravel().sum()

In [ ]:
# Aqui usamos el metodo foward fill que es llenar la obs faltante con la inmediata anteriror 
data["heartRate"]= data["heartRate"].fillna(method= "ffill") 
pd.isnull(data["heartRate"]).ravel().sum()

#con este terminamos de quitar los valores nulos del dataset

### Analisi exploratorio de los datos 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Iniciamos el analisis explorando la columna male par ver si puede afectar a las enfermedades cardiacas, pienso que la 0 es de hombres y 1 de mujeres
data["male"].value_counts() 

In [ ]:
# Podemos hacer primeramente graficas cruzadas para ver quien se enferma más 
%matplotlib inline 
pd.crosstab(data.male,data.TenYearCHD).plot(kind="bar")
plt.xlabel("Sexo")
plt.ylabel("Riesgo de enfermedad")
plt.title("Riesgo por sexo")

# este pequeño analisis arroja que es más probable que siendo mujer adquiera una enfermedad cardiaca

In [ ]:
# Analisamos si el ser o no FUMADORES puede afectar y servir para el modelo 
tabla_con_smoke= pd.crosstab(data.currentSmoker,data.TenYearCHD)
tabla_con_smoke.astype("float").div(tabla_con_smoke.sum(axis=1),axis=0)

In [ ]:
%matplotlib inline
pd.crosstab(data.currentSmoker,data.TenYearCHD).plot(kind="bar")
plt.xlabel("Fumador")
plt.ylabel("Riesgo")
# sorprendentemente no nos dice mucho si es oh  no fumador 

In [ ]:
plt.hist(data["cigsPerDay"])
plt.xlabel("cigarros por día")
plt.ylabel("Personas del estudio")

# analizando el histograma podemo concluir que el hecho de que fumen no representa un aumento significativo de enfermedades cardiacas

In [ ]:
tabla=pd.crosstab(data.BPMeds, data.TenYearCHD)
tabla.astype(float).div(tabla.sum(axis=1),axis=0)

In [ ]:
# analizamos ahora la variable bpmeds que es si el pacient tiene medicacion via intravenosa 
pd.crosstab(data.BPMeds, data.TenYearCHD).plot(kind="bar")
plt.xlabel("Medicación intravenosa")
plt.title("Riesgo de enfermedad con respecto a medicación ")

In [ ]:
#graficamos haciendo proporcionales las barras para poder ver mejor si hay mas casos si tienes medicación intravenosa, al parecer sí

table1=pd.crosstab(data.BPMeds, data.TenYearCHD)
table1.div(table1.sum(1).astype(float),axis=0).plot(kind="bar",stacked=True)
plt.xlabel("Medicacion intravenosa 0= no, 1=sí") 

In [ ]:
# ahora revisaremos prevalent stroke para ver si existio un para anteriro 
data["prevalentStroke"].hist()

In [ ]:
# prevalent hyp, si el paciente tiene hipertension 
pd.crosstab(data.prevalentHyp,data.TenYearCHD).plot(kind="bar")

In [ ]:
table3=pd.crosstab(data.prevalentHyp,data.TenYearCHD)
table3.div(table3.sum(axis=1).astype(float),axis=0).plot(kind="bar",stacked=True) 
# la hipertension definitivamente cuenta para las enfermedades del corazon

In [ ]:
# ahora analizaremos diabetes 
table4=pd.crosstab(data.diabetes,data.TenYearCHD)
table4.div(table4.sum(1),axis=0).plot(kind="bar",stacked=True) 
plt.title("Enfermedades cardiovasculares con respecto a diabetes") 

# aqui tambien analizamos la diabetes y arroja resultado tambien positivo 

In [ ]:
# corresponde el turno de colesterol(tot chol) y con este el de las variables continuas 
data["totChol"].hist() 

In [ ]:
table= pd.crosstab(data.totChol,data.TenYearCHD)
table.div(table.sum(1).astype(float),axis=0).plot(kind="bar",stacked=True)

In [ ]:
data.head()

In [ ]:
"""
Como la variable tochol es continua procedere a transformarla en una varible categorica que se dividira de la siguiente manera, si es mayor o no a 240 pues segun articulos esto se considera alto y 
puede traer concecuancias para la salud de y enfermedades cardivasculares 

https://medlineplus.gov/spanish/pruebas-de-laboratorio/niveles-de-colesterol/
"""
# usaremos la funcion tu cut para esto
categoria= pd.cut(data.totChol,bins=[0,240,696],labels=[0,1]) # cut es como lo cortara y label el valor que se le asgnaran
categoria
data.insert(6,"Colesterol",categoria) # 9 es la posicion de la nueva columna, despues el nombre y por ultimo la columna que agregaremos

In [ ]:
data.head()

In [ ]:
data.head()

In [ ]:
table= pd.crosstab(data.Colesterol,data.TenYearCHD)
table.div(table.sum(1).astype(float),axis=0).plot(kind="bar",stacked=True)

In [ ]:
table= pd.crosstab(data.Colesterol,data.TenYearCHD)
table.div(table.sum(1).astype(float),axis=0).plot(kind="bar",stacked=True)

In [ ]:
# Ahora analizaremos la varible Sys blood presurr
data["sysBP"].hist()

In [ ]:
table= pd.crosstab(data.sysBP,data.TenYearCHD)
table.div(table.sum(1).astype(float),axis=0).plot(kind="bar",stacked=True)
 

In [ ]:
# Ahora analizaremos la varible dia blood presurr
data["diaBP"].hist()

In [ ]:
table= pd.crosstab(data.diaBP,data.TenYearCHD)
table.div(table.sum(1).astype(float),axis=0).plot(kind="bar",stacked=True) 

In [ ]:
#  vemos ahora el indice de masa corporal 
data["BMI"].hist() 

In [ ]:
#como el resultado tranformaremos tambien la varible colesterol para dividrilo pasarlo a uno con más riesgo 
data["BMI"].mean() # 25.84 
data["BMI"].max() # 56.8
categoria= pd.cut(data.BMI,bins=[0,27.5,56.8],labels=[0,1]) # cut es como lo cortara y label el valor que se le asgnaran
categoria
data.insert(12,"IMC",categoria)
data.head()

In [ ]:
table=pd.crosstab(data.IMC,data.TenYearCHD)
table.div(table.sum(1).astype(float),axis=0).plot(kind="bar",stacked=True) 

In [ ]:
## vemos la relacion ahora con los latidos 
data["heartRate"].hist()  

# la variable no dice demasiado así que no se evaluara

In [ ]:
table=pd.crosstab(data.glucose,data.TenYearCHD)
table.div(table.sum(1).astype(float),axis=0).plot(kind="bar",stacked=True) 

### **Selección de variables del modelo **

In [ ]:
# Ahora quitamos la variable TenYears pues es nuestra variable a predecir
data_vars= data.columns.values.tolist()
Y= ["TenYearCHD"]
X= [v for v in data_vars if v not in Y]
X

In [ ]:
# importamos paquetes necesarios 

from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
lr= LogisticRegression() # cargamos el modelo a una variable

In [ ]:
lr= LogisticRegression() # cargamos el modelo a una variable

In [ ]:
# se usaran 12 variables para el modelo 
n=12
rfe= RFE(lr,n) #Cargamos las regresion lineal  y el numero de variables

In [ ]:
rfe= rfe.fit(data[X],data[Y].values.ravel()) # pasamos para el modelo las variables del vector X como independientes e Y como dependiente
# values.ravel es para hacer el vector Y tipo fila porque si no se pasa como vector columna

In [ ]:
print(rfe.support_) # imprimimos u a especie de vector con los resultados de que variables estaran en el modelo 

In [ ]:
print(rfe.ranking_)

In [ ]:
# hacemos un zip para crizar los datos de las columnas con los de rfe.support
z=zip(data_vars,rfe.support_)
print(z) 

In [ ]:
list(z)
cols=["male","age","BPMeds","prevalentHyp","diabetes","Colesterol","sysBP","diaBP","BMI","IMC","heartRate","glucose"]

In [ ]:
# si esta linea no se corre da este error #TypeError: cannot perform reduce with flexible type quiza porque Y no esta bien cargado o porque se carga como un tipo distindto de dato
X=data[cols]
Y=data["TenYearCHD"]

### **implementacion del modelo con Sciki-learn**

In [ ]:
from sklearn import linear_model
logistic_model2=linear_model.LogisticRegression()
logistic_model2.fit(X,Y) 

In [ ]:
logistic_model2.score(X,Y)

In [ ]:
# creamos un data frame cruzando el zip de la columna x y las cruzamos con la prediccion
pd.DataFrame(list(zip(X.columns,np.transpose(logistic_model2.coef_)))) 

### Cros validation 

In [ ]:
import numpy as np 
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y, test_size=0.3, random_state=0)

In [ ]:
# vamos a relizar el modelo con scik learn  pero con las varibles de entrenamiento 
lm=linear_model.LogisticRegression()
lm.fit(X_train,Y_train) 

In [ ]:
# Asignamos en una variable las probabilidades que seran necesarias para el modelo
probabilidades=lm.predict_proba(X_test)
#CARGAMOS paquetes para la validación
from IPython.display import display, Math, Latex
display(Math(r'Y_p=\begin{cases}0& si\ p\leq0.5\\1&si\ p >0.05\end{cases}')) # como el programa clasifica las pro por defecto
probabilidades

In [ ]:
# sacamos la predicción 
predicción=lm.predict(X_test) 
probabilidades[:,1] 
# podemos ajustar el modelo menos estricto que 0.05, creamos un nuevo array de probabilidades 
props=probabilidades[:,1] # para obtener solo la segunda columna
prob_df=pd.DataFrame(props) 
treshold=0.1
prob_df["predicción"]=np.where(prob_df[0] > treshold,1,0)
# esta linea dice si la probabilidad en la columna cero es mayor a threshold se catagolara como propenso es decir 1 y si no sera ceros es decir no propenso

In [ ]:
# calculamos cuantos caen como propensos, recurda que son los datos ya de predicción no de entrenamiento
pd.crosstab(prob_df.predicción,columns="counts")

In [ ]:
# lo vemos en porcentaje 
784/len(prob_df) *100

In [ ]:
# vamos ahora a verificar la eficacia del modelo 
from sklearn import metrics
metrics.accuracy_score(Y_test,predicción)

In [ ]:
# https://www.cienciadedatos.net/documentos/30_cross-validation_oneleaveout_bootstrap
"""
Existen varios tipos de clasificaciones pero ahora utilizaremos el K-fold-cross validation
"""
from sklearn.model_selection import cross_val_score
#scoring es la presicion de lo que necesitamos del modelo, cv es el numero de k que se dividira el modelo

In [ ]:
scores= cross_val_score(linear_model.LogisticRegression(),X,Y, scoring="accuracy",cv=10)

In [ ]:
# aqui ya hacemos predicciones con nuestros datos de entrenamiento 
scores
scores.mean() # el promedio de predicción de el modelo 

###  **Matrices de confución**  

In [ ]:
# le agregaremos un columna que seran los casos reales 
prob_df["Real"]=list(Y_test) 
prob_df

In [ ]:
#optenemos la matriz de confusion 
mc=pd.crosstab(prob_df.predicción,prob_df.Real)
mc

In [ ]:
# Sacamos el verdadero positivo
TP=mc[1][1]
TP # 158
# verdadero negativo
TN=mc[0][0] 
TN # 439
# falso negativo
FP=mc[1][0]
FP # 33
# falso negativo
FN=mc[0][1]
FN

In [ ]:
"""
optemenmos las metricas de la matriz de confusion iniciando con la exactitud o accuracy que es la proporcion 
de predicciones de enfermedad que elmodelo calsifico correctamente 
"""
Acuurracy= (TP+TN)/(TP+TN+FP+FN)
Acuurracy *100

In [ ]:
# La precisión también se conoce como valor predictivo positivo y es la proporción de instancias relevantes entre las instancias recuperadas. En otras palabras, responde a la pregunta "¿Qué proporción de identificaciones positivas fue realmente correcta?"
Presición=TP/(TP+FP)
Presición*100

In [ ]:
# sensibilidad VP/(VP+FN),  Es la proporción de casos positivos que fueron correctamente identificadas por el algoritmo. 
sensibilidad= TP/(TP+FN) 
sensibilidad # 0.20153061224489796
# Especificidad También conocida como la Tasa de Verdaderos Negativos, (“true negative rate”) o TN. Se trata de los casos negativos que el algoritmo ha clasificado correctamente.  Expresa cuan bien puede el modelo detectar esa clase. 
recall= TN/(TN+FP)
recall*100

# IDENTIFICAMOS los sanos entre los sanos